In [1]:
import os
import kagglehub
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Download the dataset
path = kagglehub.dataset_download("jagdishchakole1/mango-data")
print("Path to dataset files:", path)

# Define data directories
train_dir = os.path.join(path, "Mango_leaf_disease1/train")
val_dir = os.path.join(path, "Mango_leaf_disease1/val")
test_dir = os.path.join(path, "Mango_leaf_disease1/test")


100%|██████████| 104M/104M [00:00<00:00, 159MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/jagdishchakole1/mango-data/versions/1


In [2]:
# Create data generators with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Just rescaling for validation and test sets
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [3]:
# Set up the generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # VGG19 input size
    batch_size=16,  # Smaller batch size due to VGG19's memory requirements
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

Found 2800 images belonging to 8 classes.
Found 800 images belonging to 8 classes.
Found 400 images belonging to 8 classes.


In [4]:
# Get the number of classes
num_classes = len(train_generator.class_indices)
print(f"Number of classes: {num_classes}")
print("Class mapping:", train_generator.class_indices)


Number of classes: 8
Class mapping: {'Anthracnose': 0, 'Bacterial Canker': 1, 'Cutting Weevil': 2, 'Die Back': 3, 'Gall Midge': 4, 'Healthy': 5, 'Powdery Mildew': 6, 'Sooty Mould': 7}


In [5]:
# Create the base model from the pre-trained VGG19
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)


In [7]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Add callbacks
checkpoint = ModelCheckpoint(
    'vgg19_best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=7,
    restore_best_weights=True
)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    callbacks=[checkpoint, early_stopping]
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.4f}")

# Save the final model
model.save('mango_disease_vgg19_model.h5')

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step - accuracy: 0.3021 - loss: 2.1454

175/175 ━━━━━━━━━━━━━━━━━━━━ 74s 358ms/step - accuracy: 0.3029 - loss: 2.1427 - val_accuracy: 0.8112 - val_loss: 0.6322
Epoch 2/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - accuracy: 0.6504 - loss: 0.9598

175/175 ━━━━━━━━━━━━━━━━━━━━ 57s 323ms/step - accuracy: 0.6506 - loss: 0.9595 - val_accuracy: 0.9062 - val_loss: 0.3568
Epoch 3/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 75s 283ms/step - accuracy: 0.7742 - loss: 0.6639 - val_accuracy: 0.8888 - val_loss: 0.3458
Epoch 4/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 46s 262ms/step - accuracy: 0.7997 - loss: 0.5479 - val_accuracy: 0.8775 - val_loss: 0.3345
Epoch 5/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - accuracy: 0.8348 - loss: 0.4895

175/175 ━━━━━━━━━━━━━━━━━━━━ 107s 406ms/step - accuracy: 0.8348 - loss: 0.4894 - val_accuracy: 0.9250 - val_loss: 0.2025
Epoch 6/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 60s 282ms/step - accuracy: 0.8310 - loss: 0.4677 - val_accuracy: 0.9137 - val_loss: 0.2596
Epoch 7/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step - accuracy: 0.8407 - loss: 0.4256

175/175 ━━━━━━━━━━━━━━━━━━━━ 100s 384ms/step - accuracy: 0.8408 - loss: 0.4255 - val_accuracy: 0.9388 - val_loss: 0.1658
Epoch 8/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 60s 260ms/step - accuracy: 0.8723 - loss: 0.3632 - val_accuracy: 0.8925 - val_loss: 0.2798
Epoch 9/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 82s 262ms/step - accuracy: 0.8619 - loss: 0.3739 - val_accuracy: 0.9287 - val_loss: 0.1970
Epoch 10/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 82s 262ms/step - accuracy: 0.8736 - loss: 0.3544 - val_accuracy: 0.9225 - val_loss: 0.2095
Epoch 11/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 47s 267ms/step - accuracy: 0.8873 - loss: 0.3420 - val_accuracy: 0.9388 - val_loss: 0.1645
Epoch 12/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step - accuracy: 0.8719 - loss: 0.3411

175/175 ━━━━━━━━━━━━━━━━━━━━ 96s 347ms/step - accuracy: 0.8720 - loss: 0.3410 - val_accuracy: 0.9450 - val_loss: 0.1486
Epoch 13/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - accuracy: 0.8811 - loss: 0.3150

175/175 ━━━━━━━━━━━━━━━━━━━━ 61s 350ms/step - accuracy: 0.8811 - loss: 0.3149 - val_accuracy: 0.9638 - val_loss: 0.0962
Epoch 14/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 46s 265ms/step - accuracy: 0.9020 - loss: 0.2607 - val_accuracy: 0.9588 - val_loss: 0.1173
Epoch 15/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step - accuracy: 0.9049 - loss: 0.2816

175/175 ━━━━━━━━━━━━━━━━━━━━ 112s 438ms/step - accuracy: 0.9049 - loss: 0.2816 - val_accuracy: 0.9750 - val_loss: 0.0911
Epoch 16/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 46s 264ms/step - accuracy: 0.9040 - loss: 0.2763 - val_accuracy: 0.9688 - val_loss: 0.1035
Epoch 17/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 82s 263ms/step - accuracy: 0.9063 - loss: 0.2810 - val_accuracy: 0.9638 - val_loss: 0.1057
Epoch 18/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 82s 265ms/step - accuracy: 0.9102 - loss: 0.2475 - val_accuracy: 0.9638 - val_loss: 0.0982
Epoch 19/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 46s 264ms/step - accuracy: 0.9046 - loss: 0.2577 - val_accuracy: 0.9563 - val_loss: 0.1258
Epoch 20/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 82s 266ms/step - accuracy: 0.9151 - loss: 0.2442 - val_accuracy: 0.9588 - val_loss: 0.1042
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 135ms/step - accuracy: 0.9617 - loss: 0.1135


Test accuracy: 0.9650
